In [1]:
import os
import pandas as pd
import sys
import datetime as dt
import importlib
import pdb

sys.path.insert(1, '/users/saundo/qzanalytics/')
from qzanalytics.adapters.redshift import load_conn
from qzanalytics.usecases.system_checks import RedshiftAssist

# Google Analytics
import qzanalytics.adapters.google_analytics as GA

KEY_FILE_LOCATION = '/users/saundo/Desktop/Credentials/spherical-realm-210014-e0d0262b314f.json' 
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
VIEW_ID = '167490341'

ga = GA.GoogleAnalyticsAPI(KEY_FILE_LOCATION, SCOPES, VIEW_ID)

QS = 'query_snippet'

# pandas options
pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 100

# Redshift cost

In [ ]:
RA = RedshiftAssist()
cost = RA.spectrum_cost_report()
cost['date'] = cost['starttime'].dt.date

In [ ]:
cost.head()

In [ ]:
cost['date'].min()

In [ ]:
cost.sort_values('cost', ascending=False).iloc[1][QS]

In [9]:
sale = 1_350_000
com = 0.07
mortgage = 600_000


proceeds = sale * (1 - com)
proceeds - mortgage

655500.0

# Redshift Queries

In [4]:
query = (
    f"""
    SELECT fullvisitorid, userid
    FROM campaign_reporting.ga_events_partition
    WHERE date = '20200702'
    AND userid != 'None'
    GROUP BY fullvisitorid, userid
    """
)

conn = load_conn()
with conn as f:
    df = pd.read_sql_query(query, f)

In [5]:
df

,fullvisitorid,userid


In [ ]:
query = (
    f"""
    SELECT *
    FROM campaign_reporting.mem_dash_finance
    """
)

conn = load_conn()
with conn as f:
    df = pd.read_sql_query(query, f)

In [ ]:
df.to_csv('tableau_dash_finance.csv')

In [ ]:
df['plan_name'].value_counts()

In [ ]:
df

In [ ]:
CATEGORY = 'daily-brief – americas'

# grab all dates in 
query = (
    f"""
    WITH meta AS (
        SELECT marketing_campaign_id, marketing_campaign_name, date
        FROM campaign_reporting.email_sendgrid_meta
        WHERE category = '{CATEGORY}'
    ),
    stats AS (
        SELECT marketing_campaign_id, event, COUNT(email_id) AS ct,
               COUNT(DISTINCT email_id) AS unique
        FROM campaign_reporting.email_sendgrid_partition
        WHERE marketing_campaign_id IN (
            SELECT marketing_campaign_id FROM meta)
        GROUP BY marketing_campaign_id, event
    )
    SELECT *  
    FROM stats
    LEFT JOIN meta
    ON stats.marketing_campaign_id = meta.marketing_campaign_id
    """
)

conn = load_conn()
with conn as f:
    df = pd.read_sql_query(query, f)

In [ ]:
# grab all dates in 
query = (
    f"""
    SELECT *
    FROM svv_external_columns
    WHERE schemaname = 'spectrum'
    """
)

conn = load_conn()
with conn as f:
    df = pd.read_sql_query(query, f)
    
df.head(10)

In [2]:
# grab all dates in 
query = (
    f"""
    SELECT tablename, values, location
    FROM SVV_EXTERNAL_PARTITIONS
    WHERE schemaname = 'campaign_reporting'
    """
)

conn = load_conn()
with conn as f:
    df = pd.read_sql_query(query, f)
    
df.head(10)

,tablename,values,location
0,email_growth,"[""20200501""]",s3://qz-analytics/bigquery_ga/email_growth/date=20200501
1,email_growth,"[""20200502""]",s3://qz-analytics/bigquery_ga/email_growth/date=20200502
2,email_growth,"[""20200503""]",s3://qz-analytics/bigquery_ga/email_growth/date=20200503
3,email_growth,"[""20200504""]",s3://qz-analytics/bigquery_ga/email_growth/date=20200504
4,email_growth,"[""20200505""]",s3://qz-analytics/bigquery_ga/email_growth/date=20200505
5,email_growth,"[""20200506""]",s3://qz-analytics/bigquery_ga/email_growth/date=20200506
6,email_growth,"[""20200507""]",s3://qz-analytics/bigquery_ga/email_growth/date=20200507
7,email_growth,"[""20200508""]",s3://qz-analytics/bigquery_ga/email_growth/date=20200508
8,email_growth,"[""20200509""]",s3://qz-analytics/bigquery_ga/email_growth/date=20200509
9,email_growth,"[""20200510""]",s3://qz-analytics/bigquery_ga/email_growth/date=20200510


In [3]:
df['tablename'].value_counts()

ga_events_partition               680
ga_events_page                    315
vent_events_live                  278
gluey_2019                        227
sendgrid_test                     194
ga_events_ad_video                 78
ga_events_ad_interactions          78
ga_events_ad_impressions           78
email_growth                       72
membership_events_bq_partition     36
Name: tablename, dtype: int64

In [4]:
df[df['tablename'] == 'sendgrid_test'].tail(10)

,tablename,values,location
1748,sendgrid_test,"[""20200703""]",s3://qz-analytics/sendgrid/spectrum/partition_date=20200703
1749,sendgrid_test,"[""20200704""]",s3://qz-analytics/sendgrid/spectrum/partition_date=20200704
1750,sendgrid_test,"[""20200705""]",s3://qz-analytics/sendgrid/spectrum/partition_date=20200705
1751,sendgrid_test,"[""20200706""]",s3://qz-analytics/sendgrid/spectrum/partition_date=20200706
1752,sendgrid_test,"[""20200707""]",s3://qz-analytics/sendgrid/spectrum/partition_date=20200707
1753,sendgrid_test,"[""20200708""]",s3://qz-analytics/sendgrid/spectrum/partition_date=20200708
1754,sendgrid_test,"[""20200709""]",s3://qz-analytics/sendgrid/spectrum/partition_date=20200709
1755,sendgrid_test,"[""20200710""]",s3://qz-analytics/sendgrid/spectrum/partition_date=20200710
1756,sendgrid_test,"[""20200711""]",s3://qz-analytics/sendgrid/spectrum/partition_date=20200711
1757,sendgrid_test,"[""20200712""]",s3://qz-analytics/sendgrid/spectrum/partition_date=20200712


In [5]:
query = (
    """
    WITH temp AS (
        SELECT st.event
        FROM campaign_reporting.sendgrid_test st
        WHERE st.marketing_campaign_id IS NOT NULL
        AND partition_date = '20200712'
    )
    SELECT event, COUNT(event) as ct
    FROM temp
    GROUP BY event
    ORDER BY ct DESC
    """
)

conn = load_conn()
with conn as f:
    df1 = pd.read_sql_query(query, f)

In [6]:
df1

,event,ct
0,processed,694950
1,delivered,687125
2,open,206811
3,dropped,81935
4,deferred,29052
5,click,10985
6,bounce,10218
7,group_unsubscribe,1430
8,unsubscribe,152
9,group_resubscribe,37


In [ ]:
# grab all dates in 
query = (
    f"""
    SELECT "table", sortkey1, size, pct_used, skew_rows, vacuum_sort_benefit
    FROM SVV_TABLE_INFO
    WHERE schema = 'public'
    """
)

conn = load_conn()
with conn as f:
    df = pd.read_sql_query(query, f)
    
df.head(10)

In [ ]:
df.sort_values('size', ascending=False)

In [ ]:
# grab all dates in 
query = (
    f"""
    SELECT event_date, COUNT(event_date) as ct
    FROM sendgrid_email_campaigns
    GROUP BY event_date
    ORDER BY ct DESC
    """
)

conn = load_conn()
with conn as f:
    df = pd.read_sql_query(query, f)
    
df.info()

In [ ]:
# grab all dates in 
query = (
    f"""
    SELECT event, COUNT(event) as ct
    FROM sendgrid_email_campaigns
    WHERE event_date = '20200604'
    GROUP BY event
    ORDER BY ct DESC
    """
)

conn = load_conn()
with conn as f:
    df = pd.read_sql_query(query, f)
    
df.info()

In [ ]:
df

In [ ]:
# grab all dates in 
query = (
    f"""
    SELECT event, COUNT(event) as ct
    FROM campaign_reporting.sendgrid_test
    WHERE partition_date = '20200604'
    GROUP BY event
    ORDER BY ct DESC
    """
)

conn = load_conn()
with conn as f:
    dfx = pd.read_sql_query(query, f)

In [ ]:
dfx

In [ ]:
query = (
    """
    WITH temp AS (
        SELECT st.category,st.event,st.ip,st.marketing_campaign_id,st.marketing_campaign_name,st.sg_event_id,st.sg_message_id,st.timestamp AS event_timestamp,
               st.useragent,st.url,st.url_offset,st.marketing_campaign_split_id,st.marketing_campaign_version,st.partition_date AS event_date, eim.email_id
        FROM campaign_reporting.sendgrid_test st
        LEFT JOIN campaign_reporting.email_id_map eim
        ON st.email = eim.email
        AND st.marketing_campaign_id IS NOT NULL
        AND partition_date = '20200712'
    )
    SELECT event, COUNT(event) as ct
    FROM temp
    GROUP BY event
    ORDER BY ct DESC
    """
)

conn = load_conn()
with conn as f:
    df1 = pd.read_sql_query(query, f)

In [ ]:
df1

In [ ]:
df1

In [ ]:
# grab all dates in 
query = (
    f"""
    SELECT 
        date, 
        TO_CHAR("datetime", 'HH24:MI') as hhmm,
        COUNT(date)
    FROM ga_events_page
    WHERE datetime > '2019-12-24'
    GROUP BY date, hhmm
    """
)

conn = load_conn()
with conn as f:
    df = pd.read_sql_query(query, f)
    
df.info()

In [ ]:
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
df['dayofweek'] = df['date'].dt.dayofweek
df['today'] = '20200623 ' + df['hhmm']
df['today'] = pd.to_datetime(df['today'], format='%Y%m%d %H:%M')

In [ ]:
df.groupby('today')['count'].mean().rolling(5).mean().reset_index()

In [ ]:
max(10, 20)

In [ ]:
today = dt.datetime.now()
today = pd.to_datetime(today).dayofweek

In [ ]:
df['HHMM'] = df['date'].apply(lambda x: x.strftime('%H') + x.strftime('%M'))

In [ ]:
df['date'].dt.minute

In [ ]:
df['hours']

In [ ]:
option = 'today'
# option = 'all'

if option == 'today':
    dfx = df[df['dayofweek'] == today]
else:
    dfx = df
    
dfx = dfx.groupby(['hours', 'mins'], as_index=False)['count'].mean().rolling(5).mean().dropna()
# dfx['datetime'] = pd.to_datetime('20200622')

In [ ]:
dfx['datetime'] = '20200622:' + dfx['hours'].astype(int).astype(str) + dfx['mins'].astype(int).astype(str)


In [ ]:
date = '20200621'

st = dt.datetime.strptime(date, "%Y%m%d")
st - dt.timedelta(days=180)

In [ ]:
df['date'] = pd.to_datetime(df['date'])

# Google Analytics API

### helpful GA variable reference

https://ga-dev-tools.appspot.com/query-explorer/


### not as helpful API reference
https://developers.google.com/analytics/devguides/reporting/core/v4/rest/v4/reports/batchGet#Operator

In [ ]:
d1 = '2018-11-24'
d2 = '2018-11-24'

metrics = [
    {'expression': 'ga:pageviews'},
#     {'expression': 'ga:dfpImpressions'}
]

dimensions = [
    {'name': 'ga:dimension18'},
    {'name': 'ga:dimension26'}
]

dimension_filters = [
    {'operator': 'OR', 
     'filters':
         [{"dimensionName": "ga:dimension27",
           "not": False,
           "operator": "EXACT",
           "expressions": "1"}]
    }
]

# segments = {"segmentId": "gaid::-3"}

# run
df1 = ga.call_api(d1, d2, metrics)

df2 = ga.call_api(d1, d2, metrics, dimensions=dimensions,  
                 dimension_filters=dimension_filters,
                 segments=None)

In [ ]:
df1

In [ ]:
df2

# dicking around

In [ ]:
import datetime as dt

In [ ]:
ub = dt.datetime(2020, 5, 20, 8, 8, 50)
lb = dt.datetime(2020, 5, 20, 8, 8, 25)

(ub - lb).seconds

In [ ]:
class Query:
    _subscription_latest_invoice = (
            """
            SELECT *
            FROM (
                SELECT
                    subscription_id,
                    plan_id,
                    subscription_status,
                    current_period_start,
                    current_period_end,
                    trial_end,
                    coupon_id,
                    cancel_at_period_end,
                    latest_invoice,
                    event_timestamp,
                    ROW_NUMBER() OVER(
                        PARTITION BY subscription_id, latest_invoice
                        ORDER BY event_timestamp DESC
                        ) AS row_n
                FROM stripe_customer_subscription
                WHERE event_timestamp < 'XXMAXDATEXX'
                AND latest_invoice != 'None'
            )
            WHERE row_n = 1
            """
        )
    
    
    def init(self):
        pass
    
    @property
    def subscription_latest_invoice(self):
        """
        returns the most recent (event_timestamp) combination of:
            subscription_id
            latest_invoice

        XXMAXDATEXX -> dummy string for replacing with desired max_cutoff
        """
        
        return self._subscription_latest_invoice

In [ ]:
poop = Query()

In [ ]:

poop.subscription_latest_invoice